In [1]:
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

stopwords = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/er/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/er/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/er/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# ------- LOADING DATA INTO A DATAFRAME -------


# Loading database created by D. Beillouin et al.
xlsx_file = pd.ExcelFile(
    "/home/er/Documents/Cirad/SOCSciCompiler/data/classification_trainset/classification_trainset.xlsx"
)

# Adding label "excluded" or "included" for each MA
df_incl = xlsx_file.parse("retained_meta-analyses")
df_excl = xlsx_file.parse("non_retained_meta-analyses")
df_incl["Screening"] = "included"
df_excl["Screening"] = "excluded"

# Keeping only useful attributes
attributes_to_keep_incl = [
    "Screening",
    "link",
    "Article Title",
    "Abstract",
    "Keywords",
]
attributes_to_keep_excl = [
    "Screening",
    "lien pour accès",
    "title",
    "Abstract",
    "Keywords",
]
df_incl = df_incl[attributes_to_keep_incl]
df_excl = df_excl[attributes_to_keep_excl]

# Standardising columns names
new_column_names_incl = {"Article Title": "Title", "link": "DOI"}
new_column_names_excl = {"title": "Title", "lien pour accès": "DOI"}
df_incl = df_incl.rename(columns=new_column_names_incl)
df_excl = df_excl.rename(columns=new_column_names_excl)

# Merging exluded and included MA into single dataframe
raw_data = pd.concat([df_incl, df_excl], ignore_index=True)
raw_data = raw_data.fillna("")

size_1 = len(df_incl)
size_2 = len(df_excl)
size_3 = size_1 + size_2
print(
    f"Raw database contains {size_3} entries ({size_1} included MA and {size_2} excluded MA), stored into 'raw_data' variable."
)

Raw database contains 1007 entries (217 included MA and 790 excluded MA), stored into 'raw_data' variable.


In [3]:
# ------- CLEANING -------


# Function to get DOIs from URLs
def extract_doi(url):
    if str(url).startswith("https://doi.org/"):
        return str(url)[len("https://doi.org/") :]
    else:
        return None


# Extracting DOIs from URLs
raw_data["DOI"] = raw_data["DOI"].apply(extract_doi)

# Removing empty DOIs rows
raw_data = raw_data.dropna(subset=["DOI"])
size_4 = len(raw_data)
size_5 = size_3 - size_4
print(f"{size_5} rows removed because of empty DOIs. Cannot check the uniqueness.")

# Removing empty titles rows
raw_data["Title"] = raw_data["Title"].replace("", np.nan)
raw_data = raw_data.dropna(subset=["Title"])
size_6 = len(raw_data)
size_7 = size_4 - size_6
print(
    f"{size_7} rows removed because of empty titles. Cannot be processed by the ML model."
)

# Removing empty abstracts rows
raw_data["Abstract"] = raw_data["Abstract"].replace("", np.nan)
raw_data = raw_data.dropna(subset=["Abstract"])
size_8 = len(raw_data)
size_9 = size_6 - size_8
print(
    f"{size_9} rows removed because of empty abstracts. Cannot be processed by the ML model."
)

# Removing DOIs duplicates and titles duplicates
raw_data = raw_data.drop_duplicates(subset=["DOI"], keep="first")
raw_data = raw_data.drop_duplicates(subset="Title", keep="first")
size_10 = len(raw_data)
size_11 = size_8 - size_10
print(f"{size_11} DOI duplicates and title duplicates removed.")

# Droping column 'DOI' now we have unique values. No needed for the ML model
train_set = raw_data.drop(columns=["DOI"])

size_12 = train_set["Screening"].value_counts()
size_incl = size_12.loc["included"]
size_excl = size_12.loc["excluded"]
print(
    f"Cleaned database contains {size_10} entries ({size_incl} included MA and {size_excl} excluded MA), stored into 'train_set' variable."
)

151 rows removed because of empty DOIs. Cannot check the uniqueness.
0 rows removed because of empty titles. Cannot be processed by the ML model.
54 rows removed because of empty abstracts. Cannot be processed by the ML model.
8 DOI duplicates and title duplicates removed.
Cleaned database contains 794 entries (212 included MA and 582 excluded MA), stored into 'train_set' variable.


In [4]:
# ------- PRE-PROCESSING -------


# Function to pre-process values of columns Title, Abstract and Keywords
def preprocessor(column: str):
    token_col = []
    for val in train_set[column]:
        val = re.sub(r"[^\w\s]", "", val)  # Remove special characters/punctuation
        val = val.lower()  # Convert to lowercase
        val = val.strip()  # Remove leading/trailing spaces
        tokens = nltk.word_tokenize(val)  # Tokenization
        filtered_tokens = [
            token for token in tokens if token not in stopwords
        ]  # Stopword Removal
        lemmas = [
            lemmatizer.lemmatize(token) for token in filtered_tokens
        ]  # Lemmatization
        token_col.append(lemmas)
    train_set[column] = token_col


# Applying 'preprocessor()' to each column
preprocessor("Title")
preprocessor("Abstract")
preprocessor("Keywords")

# Shuffling and re-indexing
train_set = train_set.sample(frac=1)
train_set = train_set.reset_index(drop=True)

train_set.to_pickle('/home/er/Documents/Cirad/SOCSciCompiler/data/classification_trainset/classification_trainset.pkl')

print("Training set stored into 'train_set' variable and ready to be used.")
print("Summary with the 20 first lines:")
train_set.head(20)

Training set stored into 'train_set' variable and ready to be used.
Summary with the 20 first lines:


,Screening,Title,Abstract,Keywords
0,included,"[metaanalysis, effect, grassland, degradation,...","[alpine, meadow, qinghaitibetan, plateau, qtp,...","[alpine, meadow, degradation, community, struc..."
1,excluded,"[indirect, treatment, comparison, inotuzumab, ...","[introductionno, headtohead, study, compared, ...","[blinatumomab, indirect, treatment, comparison..."
2,excluded,"[temperature, sensitivity, soil, carbon, nitro...","[climate, warming, nitrogen, n, deposition, la...",[]
3,excluded,"[early, angiography, patient, chronic, kidney,...","[background, objective, general, population, e...",[]
4,excluded,"[nitrogen, addition, forest, soil, inhibit, de...","[enrichment, forest, soil, inorganic, nitrogen...","[lignin, decomposition, soil, carbon, cycling,..."
5,excluded,"[effect, different, agricultural, practice, ca...","[agricultural, practice, particularly, land, u...","[global, warming, potential, greenhouse, gas, ..."
6,included,"[global, metaanalysis, soil, respiration, comp...","[increasing, phosphorus, p, deposition, induce...","[soil, respiration, soil, organic, matter, dec..."
7,included,"[response, mineral, soil, carbon, storage, har...","[harvest, residue, retention, removal, influen...","[harvesting, residue, plantation, priming, eff..."
8,excluded,"[effect, transdermal, testosterone, bone, musc...","[objective, investigate, effect, testosterone,...",[]
9,included,"[effect, grazing, exclusion, carbon, sequestra...","[widespread, land, degradation, strengthened, ...",[grassland]


In [5]:
# ------- Handling Rare Words or Outliers & Vectorization -------


# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming your DataFrame is called 'df' and it contains the columns 'Title', 'Abstract', 'Keywords'

# # Step 7: Handling Rare Words or Outliers
# # Remove extremely rare words or outliers using a threshold
# threshold = 0.01  # Adjust the threshold as per your requirement
# df["Title"] = df["Title"].apply(
#     lambda x: " ".join(
#         [
#             word
#             for word in x.split()
#             if df["Title"].str.count(word).sum() / len(df) > threshold
#         ]
#     )
# )
# df["Abstract"] = df["Abstract"].apply(
#     lambda x: " ".join(
#         [
#             word
#             for word in x.split()
#             if df["Abstract"].str.count(word).sum() / len(df) > threshold
#         ]
#     )
# )
# df["Keywords"] = df["Keywords"].apply(
#     lambda x: " ".join(
#         [
#             word
#             for word in x.split()
#             if df["Keywords"].str.count(word).sum() / len(df) > threshold
#         ]
#     )
# )

# # Step 8: Vectorization
# # Perform TF-IDF vectorization on 'Title', 'Abstract', and 'Keywords'
# vectorizer = TfidfVectorizer()
# title_vectorized = vectorizer.fit_transform(df["Title"])
# abstract_vectorized = vectorizer.fit_transform(df["Abstract"])
# keywords_vectorized = vectorizer.fit_transform(df["Keywords"])

# # Convert the vectorized data into DataFrames
# title_vectorized_df = pd.DataFrame(
#     title_vectorized.toarray(), columns=vectorizer.get_feature_names()
# )
# abstract_vectorized_df = pd.DataFrame(
#     abstract_vectorized.toarray(), columns=vectorizer.get_feature_names()
# )
# keywords_vectorized_df = pd.DataFrame(
#     keywords_vectorized.toarray(), columns=vectorizer.get_feature_names()
# )

# # Concatenate the vectorized DataFrames with the original DataFrame
# df = pd.concat(
#     [df, title_vectorized_df, abstract_vectorized_df, keywords_vectorized_df], axis=1
# )

# # Display the updated DataFrame
# print(df)